### Object-Binary-Masks Orientations

In [1]:
# Install dependencies
%pip install -q pycocotools opencv-python matplotlib tqdm

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import necessary libraries and Path Setups

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from pycocotools.coco import COCO
import random

In [13]:
# Path file
ANNOTATIONS_FILE = r"/mnt/g/Authenta/data/authenta-remaskable-inpainting-detection/annotations/instances_val2017.json"
IMAGE_DIR = r"/mnt/g/Authenta/data/authenta-remaskable-inpainting-detection/images"
OUTPUT_MASKS = r"/mnt/g/Authenta/data/authenta-remaskable-inpainting-detection/masks"

In [ ]:
# Params:
MIN_MASK_AREA = 1000  # Minimum area of the mask to be considered valid
MAX_IMAGES = 50  # Maximum number of images to process

In [8]:
# Load COCO API
coco = COCO(ANNOTATIONS_FILE)
image_ids = coco.getImgIds()
print(f"Total images in dataset: {len(image_ids)}")

loading annotations into memory...
Done (t=0.60s)
creating index...
index created!
Total images in dataset: 5000


In [15]:
# helper function to create a mask for an image
def show_mask_on_image(image, mask):
    overlay = image.copy()
    overlay[mask == 1] = (0, 255, 0)
    return cv2.addWeighted(image, 0.7, overlay, 0.3, 0)


In [ ]:
#  Loop through images and extract 1 random valid mask per image
processed_count = 0

for img_id in tqdm(image_ids[:MAX_IMAGES]):
    img_info = coco.loadImgs(img_id)[0]
    img_path = os.path.join(IMAGE_DIR, img_info["file_name"])
    image = cv2.imread(img_path)

    if image is None:
        continue

    # Get valid object masks
    ann_ids = coco.getAnnIds(imgIds=img_id, iscrowd=False)
    anns = coco.loadAnns(ann_ids)

    valid_anns = []
    for ann in anns:
        if "segmentation" not in ann or type(ann["segmentation"]) != list:
            continue
        mask = coco.annToMask(ann)
        if np.sum(mask) >= MIN_MASK_AREA:
            valid_anns.append((ann, mask))

    if not valid_anns:
        continue

    #  Randomly pick one object
    ann, mask = random.choice(valid_anns)
    binary_mask = (mask * 255).astype(np.uint8)

    # Save binary mask
    mask_filename = f"mask_{img_id:012d}.png"
    mask_path = os.path.join(OUTPUT_MASKS, mask_filename)
    cv2.imwrite(mask_path, binary_mask)

    # 🔍 Optional visual check
    overlay = show_mask_on_image(image, mask)
    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    plt.title(f"{mask_filename}")
    plt.axis('off')
    plt.show()

    processed_count += 1

print(f"\n Done. Generated {processed_count} mask(s).")
